In [26]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re
import random

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

# Importar librerías para pausar la ejecución
from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [77]:
df_1 = pd.read_csv("INFO_PELIS/cortos_2000_2006.csv", index_col = 0)
df_2 = pd.read_csv("INFO_PELIS/cortos_2021.csv", index_col = 0)
df_3 = pd.read_csv("INFO_PELIS/cortos_2022.csv", index_col = 0)
df_4 = pd.read_csv("INFO_PELIS/cortos_2023.csv", index_col = 0)
df_5 = pd.read_csv("INFO_PELIS/peliculas_2021_2023.csv", index_col = 0)
df_6 = pd.read_csv("INFO_PELIS/pelis_2000_2006.csv", index_col = 0)
df_7 = pd.read_csv("INFO_PELIS/pelis_2007_2010.csv", index_col = 0)
df_8 = pd.read_csv("INFO_PELIS/pelis_2011_2013.csv", index_col = 0)
df_9 = pd.read_csv("INFO_PELIS/cortos_2007_2008.csv", index_col = 0)
df_10 = pd.read_csv("INFO_PELIS/cortos_2009_2010.csv", index_col = 0)
df_11 = pd.read_csv("INFO_PELIS/cortos_2011.csv", index_col = 0)
df_12 = pd.read_csv("INFO_PELIS/cortos_2012.csv", index_col = 0)
df_13 = pd.read_csv("INFO_PELIS/cortos_2013.csv", index_col = 0)
df_14 = pd.read_csv("INFO_PELIS/lista_pelis_2014_2019.csv", index_col = 0)
df_15 = pd.read_csv("INFO_PELIS/cortos_2014_2019.csv", index_col = 0)



In [95]:
df_15.head()

,Tipo,Titulo,Año de estreno,Mes de estreno,ID
0,Short,Le sacre d'automne,2014,11.0,tt0165933
1,Short,Eloise,2014,NaN,tt0839791
2,Short,Cocoon,2014,8.0,tt10005668
3,Short,"Zahtevam most, pripada mi!",2014,9.0,tt10010244
4,Short,Nothing is as it seems,2014,NaN,tt10010624


In [78]:
df_peliculas = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15], axis=0)

In [79]:
df_peliculas.head()

,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
0,Drama,Short,Chateaubriand - Cabeça de Paraíba,2000,12.0,tt0115874
1,Drama,Short,Nights,2000,3.0,tt0138655
2,Drama,Short,Caught,2000,8.0,tt0142155
3,Drama,Short,Smog,2000,3.0,tt0144613
4,Drama,Short,Safe Journey,2000,NaN,tt0188189


In [80]:
df_peliculas.tail()

,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
175153,NaN,Short,Temptation (Eghva),2019,9.0,tt30625971
175154,NaN,Short,Hard Way Out,2019,8.0,tt30701899
175155,NaN,Short,The Battle of the Sunset,2019,2.0,tt30766508
175156,NaN,Short,Bear Squad,2019,6.0,tt30796814
175157,NaN,Short,Dream Fast,2019,5.0,tt31186006


In [81]:
df_pelis_ordenado = df_peliculas.sort_values(by="Año de estreno")

In [82]:
df_pelis_ordenado.to_csv("pelis_ordenadas.csv")

In [84]:
lista_titulos= df_pelis_ordenado["Titulo"].to_list()

In [85]:
lista_años= df_pelis_ordenado["Año de estreno"].to_list()

In [86]:
list_busqueda_rotten = []
for titulo,año in zip(lista_titulos,lista_años):
    list_busqueda_rotten.append({titulo:año})


In [87]:
lista_tipo= df_pelis_ordenado["Tipo"].to_list()
cortos= 0
pelis = 0

for elemento in lista_tipo:
    if elemento == "Short":
        cortos += 1
    elif elemento == "Movie":
        pelis +=1

print(cortos,pelis)

#30% pelis y 70%cortos

380488 171253


In [88]:
lista_ID = df_pelis_ordenado["ID"].to_list()

In [89]:
lista1=[df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15]

suma = 0
for e in lista1:
    suma += len(e)
print("Longitus suma de los data",suma)   
print("Longitud DATAFRAME", len(lista_ID)) 


Longitus suma de los data 551741
Longitud DATAFRAME 551741


In [53]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)


## CAMBIAR A TUPLA
diccionario_peliculas = {"Puntuacion_IMDB":[], "Dirección": [],"Guionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": []}
dict_actores={}
# diccionario_actores = {"Actor":[], "Fehca_nacimiento":[], "Conocido_por:":[], "Que_hace":[], "Premios":[]}

lista_ID

for pelicula in lista_ID[130:131]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(random.randint(5,12))

    
# Seleccionar nombre de la pelicula
    try:
        diccionario_peliculas["Nombre de la película"].append(driver.find_element("css selector", 'span[data-testid="hero__primary-text"]').text)
    except:
        diccionario_peliculas["Nombre de la película"].append(f"Error nombre para la pelicula con id: {pelicula}")
        
# Seleccionar puntuacion 
    try: 
        diccionario_peliculas["Puntuacion_IMDB"].append(driver.find_element("css selector",'div[data-testid="hero-rating-bar__aggregate-rating__score"] > span:first-child').text)
        
    except:  
        diccionario_peliculas["Puntuacion_IMDB"].append("--")


    existe_direccion = False
    existe_guion = False

    for elemento in driver.find_elements("css selector",'li[data-testid="title-pc-principal-credit"]'):
 
        try:
            nombre = elemento.find_element("css selector", "span").text
            if nombre == "Dirección":
                existe_direccion = True

                # Seleccionar director
                try:          
                    diccionario_peliculas["Dirección"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                except:
                    diccionario_peliculas["Dirección"].append("DESCONOCIDO") 
            

            elif nombre == "Guión":
                existe_guion = True
            
                # Seleccionar Guionista
                try:          
                    diccionario_peliculas["Guionistas"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                except:
                    diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
        except:
            continue

    if existe_direccion == False:    
        diccionario_peliculas["Dirección"].append("DESCONOCIDO")
    if existe_guion == False:
        diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
     

    sleep(5)
    # Seleccionar ARGUMENTO
    
    try:      
        diccionario_peliculas["Argumento"].append(driver.find_element("css selector", 'span[data-testid="plot-xl"]').text)
            
    except:
        diccionario_peliculas["Argumento"].append("Desconocido")

    # Seleccionar DURACION        
    try:   
        busqueda_duracion = driver.find_element("css selector",'div.sc-491663c0-3.bdjVSf > div:first-child > ul > li:last-child').text
        if "min" in busqueda_duracion or "h" in busqueda_duracion:
            diccionario_peliculas["Duración"].append(busqueda_duracion)
        else:
            diccionario_peliculas["Duración"].append("--")
    except:
            diccionario_peliculas["Duración"].append("--")

    # Buscar los 10 primeros actores    
    try:
        for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
            actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
    
            for element in actor[0:10]:
                nombre = element.text
            
                if nombre not in dict_actores.keys():
                    dict_actores[nombre]  = element.get_attribute("href")

                    driver.find_element("css selector",'section[data-testid="title-cast"]')
    
    except:
            dict_actores[nombre]  = "desconocido"
    




### REVISAR:
# Lista de tuplas

In [90]:
diccionario_peliculas

{'Puntuacion_IMDB': ['6,2'],
 'Dirección': [['Sandro Cecca']],
 'Guionistas': [['Sandro Cecca', 'Maura Nuccetelli']],
 'Argumento': ['Life on the island of Pantelleria is becoming more difficult by the day. The ferries to the Sicilian mainland have problems running on time because of a strong wind that rises up with no warning, leaving havoc and ruin in its wake. Michele Giangrande is a rich landowner who lives in Palermo.'],
 'Duración': ['1h 30min'],
 'Nombre de la película': ['Maestrale']}

In [56]:
dict_actores

{'Franco Nero': 'https://www.imdb.com/name/nm0626259/?ref_=tt_cl_t_1',
 'Guia Jelo': 'https://www.imdb.com/name/nm0420614/?ref_=tt_cl_t_2',
 'Floriana Marino': 'https://www.imdb.com/name/nm0547769/?ref_=tt_cl_t_3',
 'Ruben Rigillo': 'https://www.imdb.com/name/nm0726765/?ref_=tt_cl_t_4',
 'Sebastiano Lo Monaco': 'https://www.imdb.com/name/nm0516241/?ref_=tt_cl_t_5',
 'Loredana Cannata': 'https://www.imdb.com/name/nm0134059/?ref_=tt_cl_t_6',
 'Gabriele Mainetti': 'https://www.imdb.com/name/nm0537734/?ref_=tt_cl_t_7',
 'Salvatore Lazzaro': 'https://www.imdb.com/name/nm0494005/?ref_=tt_cl_t_8'}

In [22]:
#Busqueda actores                 ##incluido en el codigo##

driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

# dict_actores = {"actor":[],"url":[]}
dict_actores={}


lista_ID

for pelicula in lista_ID[130:131]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(random.randint(5,12))

    for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
        actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
 
        for element in actor[0:10]:
            nombre = element.text
           
            if nombre not in dict_actores.keys():
                dict_actores[nombre]  = element.get_attribute("href")

                driver.find_element("css selector",'section[data-testid="title-cast"]')


In [24]:
dict_actores


{'Jamie Harris': 'https://www.imdb.com/name/nm0364807/?ref_=tt_cl_t_1',
 'Camilla Enders': 'https://www.imdb.com/name/nm0256813/?ref_=tt_cl_t_2',
 'Denis Holmes': 'https://www.imdb.com/name/nm0391799/?ref_=tt_cl_t_3',
 'Layla Alexander': 'https://www.imdb.com/name/nm0018916/?ref_=tt_cl_t_4',
 'Olga Alexandrova': 'https://www.imdb.com/name/nm14869118/?ref_=tt_cl_t_5',
 'Michele Ammon': 'https://www.imdb.com/name/nm1994376/?ref_=tt_cl_t_6',
 'Tyler Andrews': 'https://www.imdb.com/name/nm14869119/?ref_=tt_cl_t_7',
 'Dann R. Black': 'https://www.imdb.com/name/nm14869117/?ref_=tt_cl_t_8',
 'Valerie Bodoy': 'https://www.imdb.com/name/nm14869122/?ref_=tt_cl_t_9',
 'Jean Pierre Brax': 'https://www.imdb.com/name/nm14869114/?ref_=tt_cl_t_10'}

In [20]:
##Busqueda info actores 
driver = webdriver.Chrome()

for link_actor in dict_actores["url"]:

    link = link_actor

    driver.get(link)

    driver.maximize_window()

    sleep(5)

    # Aceptar las cookies PAG IMDB
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

    sleep(5)

    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"}
  (Session info: chrome=122.0.6261.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66A4EAD22+56930]
	(No symbol) [0x00007FF66A45F622]
	(No symbol) [0x00007FF66A3142E5]
	(No symbol) [0x00007FF66A3598ED]
	(No symbol) [0x00007FF66A359A2C]
	(No symbol) [0x00007FF66A39A967]
	(No symbol) [0x00007FF66A37BCDF]
	(No symbol) [0x00007FF66A3981E2]
	(No symbol) [0x00007FF66A37BA43]
	(No symbol) [0x00007FF66A34D438]
	(No symbol) [0x00007FF66A34E4D1]
	GetHandleVerifier [0x00007FF66A866AAD+3709933]
	GetHandleVerifier [0x00007FF66A8BFFED+4075821]
	GetHandleVerifier [0x00007FF66A8B817F+4043455]
	GetHandleVerifier [0x00007FF66A589756+70

In [91]:
df_pelis_ordenado.head()

,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
0,Drama,Short,Chateaubriand - Cabeça de Paraíba,2000,12.0,tt0115874
646,Drama,Movie,Sé quién eres,2000,4.0,tt0239955
645,Drama,Movie,La chambre obscure,2000,11.0,tt0238893
644,Drama,Movie,Shadow Magic,2000,11.0,tt0238588
643,Drama,Movie,A Christmas Tree and a Wedding,2000,11.0,tt0238121


In [93]:
df_pelis_ordenado.tail()

,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
12765,Drama,Movie,Javane Khaam,2023,3.0,tt27300783
12764,Drama,Movie,The Paternoster Gang: Streetwise,2023,4.0,tt27286412
12763,Drama,Movie,Tolani's Trial,2023,3.0,tt27280235
12779,Drama,Movie,Haywire,2023,3.0,tt27313661
1717,Drama,Short,Onvoorwaardelijk,2023,5.0,tt27495608


In [92]:
list_busqueda_rotten[:5]

[{'Chateaubriand - Cabeça de Paraíba': 2000},
 {'Sé quién eres': 2000},
 {'La chambre obscure': 2000},
 {'Shadow Magic': 2000},
 {'A Christmas Tree and a Wedding': 2000}]

In [54]:
driver = webdriver.Chrome()

url= "https://www.google.es/"

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()


sleep(3)

puntuacion_tomatoes = {"Rotten Tomatoes":[]}
contador = 0 
lista_titulos


for peli in list_busqueda_rotten[7590:7593]:
    for titulo,año in peli.items():
            
        busqueda = f"Rotten Tomatoes del {titulo} del año {año}"
        print(busqueda)


        sleep(random.randint(4,8))

        try:
            #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
        
            driver.find_element("css selector",'#APjFqb').send_keys(busqueda, Keys.ENTER)

            sleep(5)
            try:
                puntuacion_tomatoes["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:first-child > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
                # puntuacion_tomatoes["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:nth-child(3) > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
                driver.back()

                sleep(5)
                
            except:
                puntuacion_completa= (driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text)
                puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
                puntuacion_tomatoes["Rotten Tomatoes"].append(puntuacion.group())

            driver.back()
        
        except:
            puntuacion_tomatoes["Rotten Tomatoes"].append("No existe puntuacion")
        driver.back()




## MODIFICAR:
#  Se tiene que añadir al diccionario de pelis y convertir en TUPLA


Rotten Tomatoes del I've Met the Devil del año 2005
Rotten Tomatoes del Mowing the Lawn del año 2005
Rotten Tomatoes del Plastic Bitch del año 2005


In [66]:
driver = webdriver.Chrome()

url= "https://www.google.es/"

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()


sleep(3)

puntuacion_tomatoes = {"Rotten Tomatoes":[]}
lista_titulos


for peli in list_busqueda_rotten[100:103]:
    for titulo,año in peli.items():

            
        busqueda = f"Rotten Tomatoes del {titulo} del año {año}"
        print(busqueda)


        sleep(random.randint(4,8))

        try:
            
            driver.find_element("css selector",'#APjFqb').send_keys(busqueda, Keys.ENTER)
            buscador = driver.find_element("css selector",'#APjFqb')
            buscador.clear()
            buscador.send_keys(busqueda, Keys.ENTER)
            

            sleep(5)
            try:
                puntuacion_tomatoes["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:first-child > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
                # puntuacion_tomatoes["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:nth-child(3) > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
                driver.back()

                sleep(5)
                
            except:
                puntuacion_completa= (driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text)
                puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
                puntuacion_tomatoes["Rotten Tomatoes"].append(puntuacion.group())
        
        except:
            puntuacion_tomatoes["Rotten Tomatoes"].append("No existe puntuacion")
        


Rotten Tomatoes del Small Time Obsession del año 2000
Rotten Tomatoes del Les solitaires del año 2000
Rotten Tomatoes del Diary of Lust del año 2000


In [65]:
print(puntuacion_tomatoes["Rotten Tomatoes"])

['No existe puntuacion', 'No existe puntuacion', '92%']
